In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
import random
import pandas as pd
import pickle
import ast
from utilities import *
from joint_angles import JointAngle, Angles
from body_parts import BodyPart
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from statsmodels.tools.eval_measures import mse

In [2]:
fileName = '../dataset/angles_final.csv'
df = pd.read_csv(fileName)
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df['Angles'] = df['Angles'].apply(ast.literal_eval)

In [ ]:
features = df['Angles'].to_list()
labels = df['Phase'].to_list()

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

Decision Tree

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
filename = '../model/PhaseClassifier.pickle'
pickle.dump(clf, open(filename, "wb"))

In [ ]:
scores = {
    'Neck': [],
    'Left Arm': [],
    'Right Arm': [],
    'Back': [],
    'Abdomen': [],
    'Internal': [],
    'Left Leg': [],
    'Right Leg': [],
    'Angle': [],
}

In [ ]:
degrees = 180

for degree in range(degrees + 1):
    score = 0
    scores['Angle'].append(degree)
    
    for i, angle in enumerate(scores):
        if i == 8:
            continue
        delta = abs(degree - angles[i])
        score = round((1 - delta / degrees) * 100)
        scores[str(angle)].append(score)    

In [ ]:
data = {
    'Angles': [],
    'Score': []
}
for i in range(0, 1000):
    score = 0
    angles = []

    for angle in selectedAngles:
        degree = random.randint(0, 180)
        score += df[angle][degree]
        angles.append(np.array(degree))
    data['Angles'].append(np.array(angles))
    data['Score'].append(np.array(round(score / 8)))

data = pd.DataFrame(data)
# data        

In [ ]:
for i in range(0, 1000):
    if abs(data['Angles'][i][3] - data['Angles'][i][6]) > 5:
        data['Angles'][i][3] = data['Angles'][i][6] + random.randint(-4, 4)

In [ ]:
for i in range(len(data)):
    if data['Angles'][i][3] < 90 or data['Angles'][i][6] < 90:
        data['Score'][i] = 1
    else:
        data['Score'][i] = 0

In [ ]:
data = {
    'Angles': [],
    'Score': []
}

angles = []
for angle in selectedAngles:
    if 1 < df[angle].min():
        index = df[df[angle] == df[angle].min()]['Angle'].iloc[0]
    elif 1 > df[angle].max():
        index = df[df[angle] == df[angle].max()]['Angle'].iloc[0]
    else:
        index = df[df[angle] == 1]['Angle'].iloc[0]
    print(index)


In [ ]:
data = {
    'Angles': [],
    'Score': []
}

for score in range(0, 101):
    angles = []
    for angle in selectedAngles:
        if score < df[angle].min():
            index = df[df[angle] == df[angle].min()]['Angle'].iloc[0]
        elif score > df[angle].max():
            index = df[df[angle] == df[angle].max()]['Angle'].iloc[0]
        else:
            index = df[df[angle] == score]['Angle'].iloc[0]
        angles.append(np.array(index))
    angles = np.array(angles)

    data['Angles'].append(angles)
    data['Score'].append(score)

data = pd.DataFrame(data)
data
        

KMeans

In [ ]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(features)

In [ ]:
data = df.to_dict()
data['Cluster'] = kmeans.labels_
accuracy = accuracy_score(labels, data['Cluster'])
print(f"Accuracy: {accuracy * 100:.2f}%")

SVR

In [3]:
# Convert the string representation of Angles into separate columns
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

# Convert the Phase column to integers
df['Phase'] = df['Phase'].astype(int)

In [19]:
df.columns = df.columns.astype(str)

In [21]:
features = (df[['Phase', '0', '1' , '2', '3', '4', '5', '6', '7']])
labels = (df['Score'])

In [22]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [25]:
svr = svm.SVR(kernel='linear')
svr.fit(Xtrain, Ytrain)
Ypred = svr.predict(Xtest)
Ypred = Ypred.astype(int)
np.array(Ytest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 41.30%


Linear Regression

In [24]:
model = LinearRegression()
reg = model.fit(Xtrain, Ytrain)
Ypred = reg.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 16.60%


In [ ]:
filename = '../model/ScoreClassifier.pickle'
pickle.dump(svr, open(filename, "wb"))